In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.

## Troubleshooting if you have problems:

Head over to the [troubleshooting](troubleshooting.ipynb) notebook in this folder for step by step code to identify the root cause and fix it!

If you make a change, try restarting the "Kernel" (the python process sitting behind this notebook) by Kernel menu >> Restart Kernel and Clear Outputs of All Cells. Then try this notebook again, starting at the top.

Or, contact me! Message me or email ed@edwarddonner.com and we will get this to work.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point. You can also use Ollama as a free alternative, which we discuss during Day 2.

In [5]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [6]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

# Let's make a quick call to a Frontier model to get started, as a preview!

In [7]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! I'm glad to have you here. How can I assist you today?


## OK onwards with our first project

In [8]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [9]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connect
with me for

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [10]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [11]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [12]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

To give you a preview, the next 2 cells make a rather simple call - we won't stretch the might GPT (yet!)

In [11]:
# system and user prompt defined.
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [12]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Oh, the big math mystery! It’s 4. Now, what’s next? Calculus? Or are we just warming up?


## And now let's build useful messages for GPT-4o-mini, using a function

In [13]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [14]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, posi

## Time to bring it together - the API for OpenAI is very simple!

In [15]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url) # create website object
    
    # Completions API call
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [16]:
summarize("https://edwarddonner.com")

"# Summary of Edward Donner's Website\n\nThe website is run by Ed Donner, a tech enthusiast with a focus on writing code and experimenting with Large Language Models (LLMs). He is the co-founder and CTO of Nebula.io, a company utilizing AI to enhance talent discovery and engagement. Previously, he was the founder and CEO of the AI startup untapt, which was acquired in 2021. The site also lists several resources and workshops related to LLMs and AI.\n\n## Recent Announcements\n- **January 23, 2025:** LLM Workshop - Hands-on with Agents - resources\n- **December 21, 2024:** Welcome, SuperDataScientists!\n- **November 13, 2024:** Mastering AI and LLM Engineering - Resources\n- **October 16, 2024:** From Software Engineer to AI Data Scientist - resources\n\nOverall, the website serves as a hub for Ed's work and interests in technology and AI."

In [17]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [18]:
display_summary("https://edwarddonner.com")

# Website Summary: Edward Donner

Edward Donner's website serves as a personal and professional hub showcasing his interests in coding, AI, and LLMs (large language models), alongside his role as co-founder and CTO of Nebula.io. Nebula.io focuses on leveraging AI to aid talent discovery and management, offering a patented matching model and an award-winning platform.

## News and Announcements
- **January 23, 2025**: LLM Workshop – Hands-on with Agents – resources available.
- **December 21, 2024**: Welcome message for SuperDataScientists.
- **November 13, 2024**: Mastering AI and LLM Engineering – resources provided.
- **October 16, 2024**: From Software Engineer to AI Data Scientist – resources shared.

The website also includes personal insights and Ed's hobbies, such as DJing and music production.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [19]:
display_summary("https://cnn.com")

# Summary of CNN Website

CNN's website serves as a comprehensive news platform offering the latest updates across various topics including US and world news, politics, business, health, entertainment, science, and sports. The site features breaking news stories and in-depth analyses with a focus on significant ongoing events such as the **Ukraine-Russia War** and the **Israel-Hamas War**.

## Key Updates:
- **Trump Administration:** Ongoing discussions surrounding Trump’s immigration policies and a settlement agreement with Meta.
- **Legal Developments:** Former Senator Bob Menendez has been sentenced to 11 years for a corruption and bribery case.
- **Current Events:** Reports on growing unrest in Congo amid government conflicts and Israel's announcement of hostage releases.
- **Space & Science News:** Highlights include a historic asteroid sample, discoveries related to extraterrestrial origins, and insight into potential changes in space exploration.
- **Health & Wellness:** Features articles discussing the impact of social media on eating habits and recent studies on marijuana use and memory skills.

The site is structured to keep users informed through live updates, video segments, and analyses on trending topics, making it a critical resource for anyone looking to stay informed about current events globally.

In [20]:
display_summary("https://anthropic.com")

# Anthropic Website Summary

Anthropic focuses on AI research and development, emphasizing safety and alignment in artificial intelligence. The website highlights their flagship AI model, **Claude 3.5 Sonnet**, which is described as their most advanced model, now available for use. 

## Key Announcements
- **Claude 3.5 Sonnet Release**: The latest model designed for various applications.
- **Introduction of Computer Use**: Announcement regarding new features or capabilities for the Claude 3.5 Sonnet and the upcoming **Claude 3.5 Haiku**, dated **October 22, 2024**.
- **AI Safety Research**: Publications discussing the importance of AI safety and alignment, including notable works like **"Constitutional AI: Harmlessness from AI Feedback"** (December 15, 2022) and **"Core Views on AI Safety: When, Why, What, and How"** (March 8, 2023).

The website invites users to explore commercial applications of AI through the **Claude API** and details how organizations can partner with Anthropic for AI solutions. Additionally, it presents career opportunities for those interested in contributing to their mission of developing safe AI systems.

In [21]:
display_summary("https://tomoro.ai/")

# Summary of Tomoro.ai

Tomoro.ai is a company dedicated to helping businesses leverage generative AI for competitive advantage, emphasizing the transition from traditional digital solutions to advanced AI capabilities. Their approach includes:

## Core Offerings

- **Alliance**: Partnering with clients to integrate AI deeply into their operations.
- **Strategy**: Providing targeted engagements that assist clients in developing a solid AI vision, architecture, and actionable prototypes.
- **Solutions**: Developing custom AI agents that enhance knowledge management, customer service, and other innovative applications.
- **Scale**: Creating enterprise-grade AI components and collaborating with clients to accelerate the journey from ideas to scalable AI solutions.

## Autonomous AI Agents

Tomoro.ai defines autonomous AI agents as advanced collaborative solutions that operate independently, using intent-based instruction. These agents are designed to enhance business productivity by continuously learning and adapting to their environments, working alongside human teams.

## Client Engagement

The company is seeking to partner with a select number of ambitious clients who are interested in exploring these advanced AI capabilities.

For more information or to inquire about potential collaboration, interested parties can contact Tomoro.ai through a submission form on their website.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise, you experienced calling the Cloud API of a Frontier Model (a leading model at the frontier of AI) for the first time. We will be using APIs like OpenAI at many stages in the course, in addition to building our own LLMs.

More specifically, we've applied this to Summarization - a classic Gen AI use case to make a summary. This can be applied to any business vertical - summarizing the news, summarizing financial performance, summarizing a resume in a cover letter - the applications are limitless. Consider how you could apply Summarization in your business, and try prototyping a solution.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue - now try yourself</h2>
            <span style="color:#900;">Use the cell below to make your own simple commercial example. Stick with the summarization use case for now. Here's an idea: write something that will take the contents of an email, and will suggest an appropriate short subject line for the email. That's the kind of feature that might be built into a commercial email tool.</span>
        </td>
    </tr>
</table>

---
# Elevator Pitch Meeting

In [ ]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [13]:
# Step 1: Create your prompts

system_prompt = "You are an assistant at a movie studio. Your job is to take movie synopses or scripts and condense them into an elevator pitch that succintly describes what the movie is about by reframing it within the context of well known movies or commercial IPs. Respond in markdown/"

In [14]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title} which provides the synopsis for a movie"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown as well as a recontextualisation of the synopsis in the form of an elevator pitch.\n\n"
    user_prompt += website.text
    return user_prompt

In [15]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [16]:
spoiler = Website("https://themoviespoiler.com/movies/mufasa-the-lion-king/")

In [17]:
print(spoiler.title)
print(spoiler.text)

MUFASA: The Lion King – The Movie Spoiler
Skip to content
Your guide to the latest plot twists and surprise endings, now playing at a theater near you!
Primary Menu
Freestar Test Page
MUFASA: The Lion King
*CUT TO THE CHASE*
NOTE: This spoiler was submitted by
Jeremy
The film opens with the voice of the elder Mufasa telling Simba about the Circle of Life, followed by the words: “In Remembrance of James Earl Jones”.
In the kingdom of Pride Rock, Simba (Donald Glover) addresses his subjects, announcing that something great is coming (“Ngomoso”). He calls out to Nala (Beyonce Knowles-Carter), who is waiting for Simba in an oasis.
Simba leaves his daughter Kiara (Blue Ivy Carter) in the care of Timon and Pumbaa (Billy Eichner and Seth Rogen), but he tells them not to tell Kiara any wild stories. Soon as Simba leaves, they do just that by recounting the Battle of Pride Rock, except Timon and Pumbaa pretend that they were the ones who defeated Scar (and ate him). Rafiki (John Kani) arrives t

In [18]:
messages_for(spoiler)

[{'role': 'system',
  'content': 'You are an assistant at a movie studio. Your job is to take movie synopses or scripts and condense them into an elevator pitch that succintly describes what the movie is about by reframing it within the context of well known movies or commercial IPs. Respond in markdown/'},
 {'role': 'user',
  'content': 'You are looking at a website titled MUFASA: The Lion King – The Movie Spoiler\nThe contents of this website is as follows; please provide a short summary of this website in markdown as well as a recontextualisation of the synopsis in the form of an elevator pitch.\n\nSkip to content\nYour guide to the latest plot twists and surprise endings, now playing at a theater near you!\nPrimary Menu\nFreestar Test Page\nMUFASA: The Lion King\n*CUT TO THE CHASE*\nNOTE: This spoiler was submitted by\nJeremy\nThe film opens with the voice of the elder Mufasa telling Simba about the Circle of Life, followed by the words: “In Remembrance of James Earl Jones”.\nIn th

In [19]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url) # create website object
    
    # Completions API call
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [20]:
summarize("https://themoviespoiler.com/movies/mufasa-the-lion-king/")

'# Website Summary: MUFASA: The Lion King – The Movie Spoiler\n\nThe website provides a detailed synopsis of the upcoming movie "Mufasa: The Lion King." The story unfolds through the narration of elder Mufasa to his grandson Kiara, recounting the formative tales of Mufasa\'s youth, his friendships, and challenges in the animal kingdom. The film follows Mufasa’s journey from being separated from his parents due to a flood to becoming king, including his bonds with Taka (later known as Scar) and Sarabi, as well as the external threat posed by the Outsiders led by Kiros. The narrative integrates the themes of family, betrayal, and destiny, culminating in Mufasa\'s rise to kingship. \n\n# Elevator Pitch\n\nImagine *The Jungle Book* meets *The Godfather* — in "Mufasa: The Lion King," we delve into the origins of the legendary Lion King, Mufasa, as he faces betrayal from his own brother, Taka, against the backdrop of a fierce rivalry with the Outsiders. This epic tale of loyalty, love, and t

In [21]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [23]:
display_summary("https://themoviespoiler.com/movies/mufasa-the-lion-king/")

# MUFASA: The Lion King - Movie Spoiler Summary

This website provides insights into the movie "Mufasa: The Lion King," revealing plot twists and character arcs surrounding the story of Mufasa’s origin and rise to king. The film narrates how Mufasa, separated from his parents due to a flood, finds companionship with Taka and eventually faces challenges from the Outsiders. It highlights Mufasa’s journey from an outsider to a beloved king and touches on themes of brotherhood, love, and betrayal, ultimately culminating in his ascension to Pride Rock.

## Elevator Pitch
Imagine "The Lion King" meets "Kung Fu Panda" — in "Mufasa: The Lion King," we dive into the origin story of the legendary king himself. Set against a backdrop of rivalries and sibling dynamics, witness the formative journey of Mufasa as he forges bonds through adversity, confronts betrayal from his brother Taka (who ultimately becomes Scar), and rises to unite the animal kingdom against external threats. This exhilarating prequel showcases the Circle of Life in action, enriched by a familial touch that echoes the heat of Pixar’s heartfelt storytelling.

In [24]:
display_summary("https://themoviespoiler.com/movies/nosferatu/")

# Summary of NOSFERATU – The Movie Spoiler

The **NOSFERATU** website serves as a detailed guide to the film’s plot, revealing major twists and the climactic conclusion. Set in 1830s Germany, it follows the story of Ellen and her husband Thomas Hutter, who becomes ensnared by the vampire Count Orlok due to Ellen's long-standing ties to him. Orlok threatens the lives of Thomas and their friends while spreading a plague, leading to a desperate struggle to save the town and Ellen. The climax sees Ellen sacrificing herself to destroy Orlok, freeing Wisborg from his curse.

## Elevator Pitch

Imagine *Twilight* meets *The Seventh Seal*, where a young woman, Ellen, unknowingly pledged her life to a vampire when she was a child. Years later, her husband Thomas fetches a cursed castle from an ancient monster, Count Orlok, who plans to claim her once again. As Orlok spreads his plague across their town, Ellen realizes the only way to protect her loved ones is through sacrificial love. This dark reimagining of the classic tale brings horror and romance to a head in a gripping battle against the shadowy forces of the night.

In [ ]:
# Step 1: Create your prompts

system_prompt = "something here"
user_prompt = """
    Lots of text
    Can be pasted here
"""

# Step 2: Make the messages list

messages = [] # fill this in

# Step 3: Call OpenAI

response =

# Step 4: print the result

print(

## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. In the community-contributions folder, you'll find an example Selenium solution from a student (thank you!)

# Sharing your code

I'd love it if you share your code afterwards so I can share it with others! You'll notice that some students have already made changes (including a Selenium implementation) which you will find in the community-contributions folder. If you'd like add your changes to that folder, submit a Pull Request with your new versions in that folder and I'll merge your changes.

If you're not an expert with git (and I am not!) then GPT has given some nice instructions on how to submit a Pull Request. It's a bit of an involved process, but once you've done it once it's pretty clear. As a pro-tip: it's best if you clear the outputs of your Jupyter notebooks (Edit >> Clean outputs of all cells, and then Save) for clean notebooks.

Here are good instructions courtesy of an AI friend:  
https://chatgpt.com/share/677a9cb5-c64c-8012-99e0-e06e88afd293